In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

In [2]:
from src.models.modules import *
from src.models.loss import L1_epsilon_lambda
from dataclasses import dataclass
import torch

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

@dataclass
class SDFTransformerConfig:
    dim_context: int
    dim_input: int
    num_outputs: int
    dim_output: int
    delta: float = 0.1
    dim_hidden: int = 64
    num_inds: int = 32
    num_heads: int = 4
    ln: bool = False

class SDFEncoder(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFEncoder, self).__init__()
        self.config = config

    def forward(self, context: torch.Tensor):
        return torch.ones(context.shape[0], config.num_outputs, config.dim_hidden).to(device)

class SDFDecoder(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFDecoder, self).__init__()
        self.dec = nn.Sequential(
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            nn.Linear(config.dim_hidden, config.dim_output),
            nn.Tanh()
        )

    def forward(self, context: torch.Tensor):
        return self.dec(context)

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.epsilon = None
        self.lambdaa = None
        self.enc = SDFEncoder(config)
        self.input_proj = nn.Linear(config.dim_input, config.dim_hidden)
        self.cross = MAB(config.dim_hidden, config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln)
        self.dec = SDFDecoder(config)

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        y = self.enc(context)           # [batch_size, context_size, dim_hidden]
        x = self.input_proj(x)          # [batch_size, num_outputs, dim_hidden]
        x = x.repeat(1, y.shape[1], 1)  # [batch_size, context_size, dim_hidden]
        y = self.cross(x, y)            # [batch_size, context_size, dim_hidden]
        y = self.dec(y)                 # [batch_size, num_outputs, dim_output]

        loss = None
        if labels is not None:
            loss = L1_epsilon_lambda(y, labels, self.epsilon, self.lambdaa, self.config.delta)
        return {'loss': loss, 'logits': y}

config = SDFTransformerConfig(dim_context=4, dim_input=3, num_outputs=1, dim_output=1)
model = SDFTransformer(config).to(device)
print(device)

cuda


In [3]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [ ]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2024_12_06_no_enc'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2024_12_06_enhanced_encoder-2024-12-08-20-17-19


In [5]:
from transformers import Trainer, TrainingArguments

batch_size = 40
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=3,
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

curriculum_schedule = [
    {"epochs": 2, "epsilon": 0.02,   "lambda": 0.0,  'learning_rate': 5e-5},
    {"epochs": 2, "epsilon": 0.0075, "lambda": 0.15, 'learning_rate': 4e-5},
    {"epochs": 2, "epsilon": 0.004,  "lambda": 0.3,  'learning_rate': 3e-5},
    {"epochs": 2, "epsilon": 0.002,  "lambda": 0.4,  'learning_rate': 2e-5},
    {"epochs": 2, "epsilon": 0.0,    "lambda": 0.5,  'learning_rate': 1e-5}
]

In [6]:
from src.visualization.generate_mesh import generate_meshes
from src.data.load_data import get_data_dir

obj_dir = get_data_dir() / 'intermediate'
format_string_base = "{name}-" + current_date + "-curriculum-"

for i, stage in enumerate(curriculum_schedule):
    model.epsilon = stage['epsilon']
    model.lambdaa = stage['lambda']
    trainer.args.num_train_epochs = stage['epochs']
    trainer.args.learning_rate = stage['learning_rate']
    trainer.train()
    format_string = format_string_base + str(i) + ".obj"
    generate_meshes(model, obj_dir, result_dir, format_string, device,
        batch_size, resolution=100, context_size=200)
train_dataset.close()
val_dataset.close()

  0%|          | 0/18000 [00:00<?, ?it/s]

{'loss': 0.0028, 'grad_norm': 0.08662301301956177, 'learning_rate': 4.997222222222223e-05, 'epoch': 0.0}
{'loss': 0.0037, 'grad_norm': 0.0, 'learning_rate': 4.994444444444445e-05, 'epoch': 0.0}
{'loss': 0.0044, 'grad_norm': 0.043719545006752014, 'learning_rate': 4.991666666666667e-05, 'epoch': 0.0}
{'loss': 0.0031, 'grad_norm': 0.08595050871372223, 'learning_rate': 4.9888888888888894e-05, 'epoch': 0.0}
{'loss': 0.002, 'grad_norm': 0.04164993762969971, 'learning_rate': 4.986111111111111e-05, 'epoch': 0.01}
{'loss': 0.0039, 'grad_norm': 0.010827702470123768, 'learning_rate': 4.9833333333333336e-05, 'epoch': 0.01}
{'loss': 0.003, 'grad_norm': 0.04617235064506531, 'learning_rate': 4.9805555555555554e-05, 'epoch': 0.01}
{'loss': 0.0032, 'grad_norm': 0.17770326137542725, 'learning_rate': 4.977777777777778e-05, 'epoch': 0.01}
{'loss': 0.0028, 'grad_norm': 0.08496013283729553, 'learning_rate': 4.975e-05, 'epoch': 0.01}
{'loss': 0.0056, 'grad_norm': 0.2159336656332016, 'learning_rate': 4.972222

  0%|          | 0/1000 [00:00<?, ?it/s]

{'eval_loss': 0.0012926532654091716, 'eval_runtime': 21.9197, 'eval_samples_per_second': 1824.84, 'eval_steps_per_second': 45.621, 'epoch': 1.0}
{'loss': 0.0016, 'grad_norm': 0.06525403261184692, 'learning_rate': 2.4972222222222226e-05, 'epoch': 1.0}
{'loss': 0.002, 'grad_norm': 0.035227082669734955, 'learning_rate': 2.4944444444444447e-05, 'epoch': 1.0}
{'loss': 0.0013, 'grad_norm': 0.07954756915569305, 'learning_rate': 2.4916666666666668e-05, 'epoch': 1.0}
{'loss': 0.0018, 'grad_norm': 0.06144788861274719, 'learning_rate': 2.488888888888889e-05, 'epoch': 1.0}
{'loss': 0.0011, 'grad_norm': 0.11638706922531128, 'learning_rate': 2.4861111111111114e-05, 'epoch': 1.01}
{'loss': 0.0008, 'grad_norm': 0.1395125538110733, 'learning_rate': 2.4833333333333335e-05, 'epoch': 1.01}
{'loss': 0.0004, 'grad_norm': 0.0, 'learning_rate': 2.4805555555555556e-05, 'epoch': 1.01}
{'loss': 0.0011, 'grad_norm': 0.0727742537856102, 'learning_rate': 2.477777777777778e-05, 'epoch': 1.01}
{'loss': 0.0012, 'grad_

  0%|          | 0/1000 [00:00<?, ?it/s]

...

{'eval_loss': 0.004394472111016512, 'eval_runtime': 23.5115, 'eval_samples_per_second': 1701.292, 'eval_steps_per_second': 42.532, 'epoch': 1.0}
{'loss': 0.0053, 'grad_norm': 0.2139846384525299, 'learning_rate': 2.4972222222222226e-05, 'epoch': 1.0}
{'loss': 0.0058, 'grad_norm': 0.5842447280883789, 'learning_rate': 2.4944444444444447e-05, 'epoch': 1.0}
{'loss': 0.0048, 'grad_norm': 0.5438944101333618, 'learning_rate': 2.4916666666666668e-05, 'epoch': 1.0}
{'loss': 0.0057, 'grad_norm': 0.27059099078178406, 'learning_rate': 2.488888888888889e-05, 'epoch': 1.0}
{'loss': 0.0047, 'grad_norm': 0.5067827701568604, 'learning_rate': 2.4861111111111114e-05, 'epoch': 1.01}
{'loss': 0.0035, 'grad_norm': 0.4927249252796173, 'learning_rate': 2.4833333333333335e-05, 'epoch': 1.01}
{'loss': 0.0026, 'grad_norm': 0.41908150911331177, 'learning_rate': 2.4805555555555556e-05, 'epoch': 1.01}
{'loss': 0.0038, 'grad_norm': 1.4401429891586304, 'learning_rate': 2.477777777777778e-05, 'epoch': 1.01}
{'loss': 0.

  0%|          | 0/1000 [00:00<?, ?it/s]

{'eval_loss': 0.0040221489034593105, 'eval_runtime': 22.7268, 'eval_samples_per_second': 1760.035, 'eval_steps_per_second': 44.001, 'epoch': 2.0}
{'train_runtime': 698.5507, 'train_samples_per_second': 1030.705, 'train_steps_per_second': 25.768, 'train_loss': 0.004408453373031484, 'epoch': 2.0}


Processing models:   0%|          | 0/1 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]

In [7]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)